# AIS DECODER

In [1]:
#aisdecoder env
import pandas as pd
import geopandas as gpd
from decoder import NMEA_decoder
from datetime import date
from shapely.geometry import Point, Polygon
import folium 
import numpy as np



from mapper import *

today = date.today().strftime("%Y-%m-%d")

style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

In [2]:
file_path = "SRC\Input\Vesperino_24032022.txt"
ais_msg = NMEA_decoder(file_path)
GPS = ais_msg.get_GPS()
ais = ais_msg.to_df()
# ais.to_excel('SRC\\Output\\131021\\Ais_decoded.xlsx')
# GPS.to_excel('SRC\\Output\\131021\\GPS_decoded.xlsx')
ais.dropna(axis=0,subset=['lon','lat'], inplace=True)
ais.reset_index(inplace=True, drop=True)

# MAPPER


In [3]:
style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

# output_path = 'SRC\\Output\\'
# decoded_path = '071021'

# ais = pd.read_excel(output_path+decoded_path+'\\'+'Ais_decoded.xlsx')
# ais.drop(ais.filter(regex='Unname'), axis=1, inplace=True)
# ais=ais[ais['lon'].notnull()]
# ais.reset_index(drop=True, inplace=True)

# GPS = pd.read_excel(output_path+decoded_path+'\\'+'GPS_decoded.xlsx')
# GPS.drop(GPS.filter(regex='Unname'), axis=1, inplace=True)

In [6]:
ais

,timestamp,type,repeat,mmsi,status,turn,speed,accuracy,lon,lat,...,band,msg22,partno,vendorid,model,serial,mothership_mmsi,seqno,dest_mmsi,retransmit
0,16:55:42,1,0,247295700,0.0,-128.0,5.6,0.0,14.299650,40.828633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16:55:43,1,0,247047200,0.0,-128.0,14.7,0.0,14.231333,40.800733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16:55:44,1,0,247171200,0.0,-128.0,0.1,0.0,14.268243,40.841195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16:55:44,1,0,636016492,0.0,0.0,7.2,0.0,14.299947,40.827427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16:55:45,1,0,247437300,0.0,0.0,0.1,1.0,14.264707,40.843598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,16:59:33,21,0,992471042,NaN,NaN,NaN,0.0,14.272122,40.832847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,16:59:33,1,0,636016492,0.0,0.0,6.9,0.0,14.292757,40.832332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,16:59:34,1,0,247376900,5.0,-127.0,6.8,0.0,14.291587,40.832818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,16:59:34,1,0,247430300,5.0,0.0,7.0,0.0,14.293567,40.831817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ais = ais[ (ais['timestamp']>'16:55:39') & (ais['timestamp']<'16:59:39')]
# ais = ais.drop_duplicates(subset='mmsi')
ais.dropna(axis=0,subset=['lon','lat'], inplace=True)
ais.reset_index(inplace=True, drop=True)

C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/88278477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ais.dropna(axis=0,subset=['lon','lat'], inplace=True)


# Create Shapefiles

In [8]:
SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])
# ais = ais[ (ais['timestamp']>'16:44:59') & (ais['timestamp']<'16:59:24')]
ais.reset_index(inplace=True, drop=True)

output_path = 'SRC\\Output\\'
decoded_path = '240322'

for i in range(len(ais)):
    label = ais['mmsi'][i]
    # lat, lon = ais['latitude'][i], ais['longitude'][i]
    lat, lon = float(ais['lat'][i]), float(ais['lon'][i])
    coordinates = [(lon, lat)]
    geometry = Point(coordinates)
    new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry}
    SHP = SHP.append(new_row, ignore_index=True)
SHP.to_file(output_path+decoded_path+'\\'+'navi.shp')

C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/1437513077.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/1437513077.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/1437513077.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/1437513077.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_

### GPS

In [ ]:
GPS.head(3)

In [7]:
SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])
GPS = GPS[ (GPS['timestamp']>'16:56:00') & (GPS['timestamp']<'16:57:00')]
GPS.reset_index(inplace=True, drop=True)

output_path = r'SRC\Output\240322'
# decoded_path = '131021'

for i in range(len(GPS)):
    label = GPS['timestamp'][i]
    lat, lon = GPS['latitude'][i], GPS['longitude'][i]
    coordinates = [(lon, lat)]
    geometry = Point(coordinates)
    new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry}
    SHP = SHP.append(new_row, ignore_index=True)
SHP.to_file(output_path+'GPS.shp')

C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/3874609642.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/3874609642.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/3874609642.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_index=True)
C:\Users\ROBE~1.DEL\AppData\Local\Temp/ipykernel_20368/3874609642.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SHP = SHP.append(new_row, ignore_

In [ ]:
SHP

# Create HTML

In [ ]:
ships_mmsi = ais['mmsi'].unique() # mmsi of ships identified
n_ships_mmsi = ais['mmsi'].nunique() # number of ships identified


for mmsi in ships_mmsi:
     PlotPolyline(m, SubsetAis_mmsi(mmsi, ais))

# plotReceiver(GPS.iloc[0:1], m)
points = np.arange(0,GPS.shape[0], 100) # Sampling 20 sec

for point in points:
     plotReceiver(GPS.iloc[point:point+1], m)


add_my_legend(m)
m
# df = SubsetAis_mmsi(ships_mmsi[3], ais)
# points = [(x.lat, x.lon) for x in df.iloc]
# points

In [ ]:
m.save('map.html')